In [1]:
from IPython.display import display, HTML
# display(HTML("<style>.container { width:95% !important; }</style>"))

import mimikit as mmk
import ipywidgets as W
import dataclasses as dtc
from typing import Optional, Callable, Any, Tuple, List, Dict
from IPython.display import display

UI = mmk.ui

data_cfg = mmk.DatasetConfig()

display(UI.MMK_STYLE_SHEET, mmk.views.dataset_view(data_cfg))

HTML(value='\n<style>\n\n.picker-button {\n    border: 0px !important;\n    border-radius: 5px !important;\n  …

DatasetView(children=(VBox(children=(Tab(children=(AppLayout(children=(HTML(value='<h4>Select Soundfiles</h4>'…

In [2]:
tr_cfg = mmk.TrainARMConfig()

NETWORKS = dict(
    wavenet=(mmk.WaveNet, mmk.WaveNet.Config(), mmk.wavenet_view),
    sample_rnn=(mmk.SampleRNN, mmk.SampleRNN.Config(), mmk.sample_rnn_view)
)

IO = dict(
    mulaw=(mmk.IOSpec.MuLawIOConfig(), mmk.mulaw_io_view),
    yt=(mmk.IOSpec.YtIOConfig(), mmk.yt_io_view),
    magspec=(mmk.IOSpec.MagSpecIOConfig(), mmk.magspec_io_view)
)

select_net = UI.EnumWidget(
    "select network class: ",
    list(NETWORKS.keys()),
    selected_index=0
)

select_io = UI.EnumWidget(
    "select input/target type: ",
    list(IO.keys()),
    selected_index=0
)

def run(ev):
    net_cls, net_cfg, _ = NETWORKS[select_net.value]
    net = net_cls.from_config(net_cfg)
    db = data_cfg.get(mode='r', keep_open=True)
    hp = mmk.ARMHP(data_cfg, net_cfg, tr_cfg)
    loop = mmk.TrainARMLoop.from_config(hp, db, net)
    loop.run()


submit = W.Button(description="RUN", layout=W.Layout(width="90%"))
submit.on_click(run)

_, cfg, view_func = NETWORKS['wavenet']
io_cfg, io_view = IO["mulaw"]
container = W.VBox([
    select_io,
    io_view(io_cfg),
    select_net,
    view_func(cfg),
    mmk.views.train_arm_view(tr_cfg), submit
])

def display_net_config(ev):
    _, cfg, view_func = NETWORKS[ev["new"]]
    container.children = (*container.children[:2], select_net, view_func(cfg), *container.children[4:])
    
select_net.observe(display_net_config, 'value')

def display_io_config(ev):
    cfg, view_func = IO[ev["new"]]
    container.children = (select_io, view_func(cfg), *container.children[2:])
    
select_io.observe(display_io_config, 'value')


display(container)

select_net.selected_index = 0
select_io.selected_index = 0

In [3]:
c, v = IO["yt"]
v(c)

Accordion()

In [2]:



train_view = mmk.ConfigView(
    cfg,
    # ---------------------------
    UI.Param(
        name="data",
        widget=),
    # ---------------------------
    UI.Param(
        name="network",
        widget=),
    # ---------------------------
    UI.Param(
        name="training",
        widget=)
    # Layout
    # Config Validation
)



display(*train_view.widgets)

SyntaxError: invalid syntax (<ipython-input-2-8c80a16fa9b4>, line 6)

In [ ]:
mmk.demos.freqnet.main(cfg)